In [1]:
! pip install -U git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-a372_83t
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-a372_83t
  Stored in directory: /tmp/pip-ephem-wheel-cache-nm9cnufp/wheels/64/60/2e/30ebaa76ed1626e86bfb0cc0579b737fdb7d9ff8cb9522663a
Successfully built efficientnet
  Found existing installation: efficientnet 1.0.0
    Uninstalling efficientnet-1.0.0:
      Successfully uninstalled efficientnet-1.0.0


In [2]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks.callbacks import ReduceLROnPlateau, ModelCheckpoint

from keras.applications.imagenet_utils import decode_predictions

import efficientnet.tfkeras as efn 
from efficientnet.keras import center_crop_and_resize, preprocess_input



Using TensorFlow backend.


19 klas
['0Parus', '1Passe', '2Lusci', '3Phoen', '4Erith', '5Phoen', '6Sitta', '7Alaud', '8Phyll', '9Turdu', '10Phyll', '11Fring', '12Sturn', '13Ember', '14Colum', '15Trogl', '16Cardu', '17Chlor', '18Turdu']
16 klas
['0Parus', '1Passe', '2Lusci',
               '3Phoen', '4Erith', '5Phoen',
               '6Sitta', '7Alaud', '8Phyll',
                '9Turdu', '10Phyll', '11Fring',
               '13Ember',
               '15Trogl', '17Chlor', '18Turdu']

In [3]:
IM_SIZE = (224,224)
birds_classes=['0Parus', '1Passe', '2Lusci', '3Phoen', '4Erith', '5Phoen', '6Sitta', '7Alaud', '8Phyll', '9Turdu',
               '10Phyll', '11Fring', '12Sturn', '13Ember', '14Colum', '15Trogl', '16Cardu', '17Chlor', '18Turdu']
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory('data/19_class/train',
                                                  classes=birds_classes,
                                                  target_size=IM_SIZE,
                                                  class_mode='categorical', shuffle=True,
                                                  batch_size=16)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory('data/19_class/val',
                                                  classes=birds_classes,
                                                  target_size=IM_SIZE,
                                                  class_mode='categorical', shuffle=False,
                                                  batch_size=16)

# show class indices
print('****************')
for cls, idx in train_batches.class_indices.items():
    print('Class nr ',idx,' -> ', cls)
print('****************')


Found 118716 images belonging to 19 classes.
Found 13972 images belonging to 19 classes.
****************
Class nr  0  ->  0Parus
Class nr  1  ->  1Passe
Class nr  2  ->  2Lusci
Class nr  3  ->  3Phoen
Class nr  4  ->  4Erith
Class nr  5  ->  5Phoen
Class nr  6  ->  6Sitta
Class nr  7  ->  7Alaud
Class nr  8  ->  8Phyll
Class nr  9  ->  9Turdu
Class nr  10  ->  10Phyll
Class nr  11  ->  11Fring
Class nr  12  ->  12Sturn
Class nr  13  ->  13Ember
Class nr  14  ->  14Colum
Class nr  15  ->  15Trogl
Class nr  16  ->  16Cardu
Class nr  17  ->  17Chlor
Class nr  18  ->  18Turdu
****************


In [4]:
ModelCheck = ModelCheckpoint('models/AM_efficientnet_19classes_callbackb4.h5', monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1)
ReduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=3e-4)

net = efn.EfficientNetB4(include_top=False,
                        weights='imagenet',
                          input_tensor=None,
                        input_shape=(224,224,3))
x = net.output
x = Flatten()(x)
x = Dropout(0.4)(x)
output_layer = Dense(19, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:20]:
    layer.trainable = False
for layer in net_final.layers[20:]:
    layer.trainable = True
net_final.compile(optimizer=Adam(),
                  loss='categorical_crossentropy', metrics=['accuracy'])
print(net_final.summary())



71892992/71892840 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 48) 0           stem_bn[0][0]                    
_____________________________

In [ ]:
import numpy as np

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_batches.classes), 
                train_batches.classes)
class_weights
# train the model
net_final.fit_generator(train_batches,
                        validation_data = valid_batches,
                        epochs = 20,
                        steps_per_epoch= 1000,
                        class_weight=class_weights, callbacks=[ModelCheck,ReduceLR])


Epoch 1/20
1000/1000 [==============================] - 8982s 9s/step - loss: 8.6799 - accuracy: 0.3061 - val_loss: 11.6873 - val_accuracy: 0.2403
Epoch 2/20
 999/1000 [============================>.] - ETA: 7s - loss: 9.7272 - accuracy: 0.2659 